<a href="https://colab.research.google.com/github/lNamelessl/ML-Projects/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle/

# Now copy the kaggle.json file from /content/kaggle.json
!cp /content/kaggle.json ~/.kaggle/

# And set the permissions
!chmod 600 ~/.ka

chmod: cannot access '/root/.ka': No such file or directory


In [ ]:
# download the dataset from kaggle
!kaggle competitions download -c fake-news

In [ ]:
# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle/

# Now copy the kaggle.json file from /content/kaggle.json
!cp /content/kaggle.json ~/.kaggle/

# And set the permissions
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# unzip the dataset
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd

In [ ]:
# load the training df
train_df = pd.read_csv("train.csv")

In [ ]:
# load the testing df
test_df = pd.read_csv("test.csv")

In [ ]:
# check the datasets
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# check the datasets
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
# check for missing values
train_df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
# check for missing values
test_df.isna().sum()

,0
id,0
title,122
author,503
text,7


In [ ]:
import nltk
import spacy
import string
from nltk.corpus import stopwords

In [ ]:
# download stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = stopwords.words("english")

In [ ]:
# load the spacy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# define preprocessing function
def preprocess(text):
  if pd.isna(text):
    return ""

  # convert to lowercase
  text = text.lower()

  # Remove punctuations
  text = text.translate(str.maketrans("", "", string.punctuation))

  # tokenization and limitization using spacy
  doc  = nlp(text)
  tokens = [token.lemma_ for token in doc if token.text not in stopwords]

  return " ".join(tokens)


In [ ]:
train_df["cleaned_text"] = train_df["text"].apply(preprocess)

In [ ]:
# check the results
train_df.head()

,id,title,author,label,cleaned_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,1,house dem aide not even see comey ’s letter ja...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,0,ever get feeling life circle roundabout rather...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,1,truth might get fire october 29 2016 \n tensio...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,1,video 15 civilian kill single us airstrike ide...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,1,print \n iranian woman sentence six year priso...


In [ ]:
train_df.drop(columns=["text"], inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
# Initialize the vectorizer
tfidf = TfidfVectorizer(max_features=5000)


In [ ]:
# transform the  text data
x_tfidf = tfidf.fit_transform(train_df["cleaned_text"])


In [ ]:
# convert to df
x_tfidf_df = pd.DataFrame(x_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [ ]:
# check the shape
x_tfidf_df.shape

(20800, 5000)

In [ ]:
from sklearn.feature_selection import  chi2, SelectKBest


In [ ]:
# Select top 2000 features based on Chi-square test
selector = SelectKBest(chi2, k=2000)
x_selected = selector.fit_transform(x_tfidf_df, train_df["label"])

In [ ]:
# Get selected feature names out
selected_features = x_tfidf_df.columns[selector.get_support()]
print(f"Selected {len(selected_features)}features")

Selected 2000features


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(x_selected, train_df["label"], test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [ ]:

# Define models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
    "SVM": SVC(kernel="linear"),
    "Naive Bayes": MultinomialNB()
}

# Train & Evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")


Random Forest Accuracy: 0.9502


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:44:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.9620
SVM Accuracy: 0.9500
Naive Bayes Accuracy: 0.9026


In [ ]:
best_model = models["XGBoost"]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.88      0.94      0.91      2132
           1       0.93      0.86      0.90      2028

    accuracy                           0.90      4160
   macro avg       0.91      0.90      0.90      4160
weighted avg       0.90      0.90      0.90      4160

[[2007  125]
 [ 280 1748]]


In [ ]:
import shap

In [ ]:
import joblib

joblib.dump(best_model, "final_model.pkl")


['final_model.pkl']